Version: 02.14.2023

# Lab 2.1 - Menerapkan ML untuk Masalah NLP

Di lab ini, Anda akan menggunakan model <i>machine learning</i> (ML) bawaan di Amazon SageMaker, __LinearLearner__, untuk memprediksi bidang __isPositive__ dari set data ulasan.


## Pengantar skenario bisnis
Anda bekerja untuk toko ritel online yang ingin meningkatkan keterlibatan pelanggan bagi pelanggan yang telah memposting ulasan negatif. Perusahaan ingin mendeteksi ulasan negatif dan menetapkan ulasan ini ke agen layanan pelanggan untuk diatasi.

Anda ditugaskan untuk memecahkan bagian dari masalah ini dengan menggunakan ML untuk mendeteksi ulasan negatif. Anda diberi akses ke set data yang berisi ulasan, yang telah diklasifikasikan sebagai positif atau negatif. Anda akan menggunakan set data ini untuk melatih model ML untuk memprediksi sentimen ulasan baru.

## Tentang set data ini
File [AMAZON-REVIEW-DATA-CLASSIFICATION.csv] (https://github.com/aws-samples/aws-machine-learning-university-accelerated-nlp/tree/master/data/examples) berisi ulasan aktual produk, dan ulasan ini mencakup data teks dan data numerik. Setiap ulasan diberi label sebagai _positive (1)_ atau _negative (0)_.


Set data berisi fitur berikut:
* __reviewText:__ Teks ulasan
* __summary:__ Ringkasan ulasan
* __verified:__ Apakah pembelian telah diverifikasi (Benar atau Salah)
* __time:__ <i>Timestamp</i> UNIX untuk ulasan
* __log_votes:__ Log <i>vote</i> yang disesuaikan algoritme(1+<i>vote</i>)
* __isPositive:__ Apakah ulasan positif atau negatif (1 atau 0)

Set data untuk lab ini disediakan untuk Anda dengan izin dari Amazon dan tunduk pada persyaratan Lisensi dan Akses Amazon (tersedia di https://www.amazon.com/gp/help/customer/display.html?nodeId=201909000). Anda secara tegas dilarang menyalin, memodifikasi, menjual, mengekspor, atau menggunakan set data ini dengan cara apa pun selain untuk tujuan menyelesaikan kursus ini.

## Langkah-langkah lab

Untuk menyelesaikan lab, Anda akan mengikuti langkah-langkah berikut:

1. [Membaca set data](#1.-Reading-the-dataset)
2. [Melakukan analisis data eksplorasi](#2.-Performing-exploratory-data-analysi)
3. [Pemrosesan teks: Menghapus <i>stopwords </i>dan <i>stemming</i>](#3.-Pemrosesan teks:-Removing-stopwords-and-stemming)
4. [Memisahkan data pelatihan, validasi, dan tes](#4.-Splitting-training,-validation,-and-test-data)
5. [Memproses data dengan alur dan ColumnTransformer](#5.-Processing-data-with-pipelines-and-a-ColumnTransformer)
6. [Melatih pengklasifikasi dengan algoritma SageMaker bawaan](#6.-Training-a-classifier-with-a-built-in-SageMaker-algorithm)
7. [Mengevaluasi model](#7.-Evaluating-the-model)
8. [Men-<i>deploy </i>model ke titik akhir](#8.-Deploying-the-model-to-an-endpoint)
9. [Menguji titik akhir](#9.-Testing-the-endpoint)
10. [Membersihkan artifacts model](#10.-Cleaning-up-model-artifacts)
    
## Mengirimkan pekerjaan Anda

1. Di konsol lab, pilih **<i>Submit</i>** (Kirim) untuk merekam kemajuan Anda dan saat diminta, pilih **<i>Yes</i>** (Ya).

1. Jika hasilnya tidak muncul setelah beberapa menit, kembali ke bagian atas petunjuk ini dan pilih <i>**Grades**</i> (Nilai).

     **Tip**: Anda dapat mengirimkan pekerjaan Anda beberapa kali. Setelah Anda mengubah pekerjaan Anda, pilih **<i>Submit</i>** (Kirim) lagi. Kiriman terakhir Anda akan direkam untuk lab ini.

1. Untuk menemukan detail umpan balik tentang pekerjaan Anda, pilih <i>**Details** </i>(Detail) yang diikuti dengan **<i>View Submission Report</i>** (Lihat Laporan Pengiriman).  

Mulailah dengan menginstal/memutakhirkan pip, sagemaker, dan scikit-learn.

[scikit-learn](https://scikit-learn.org/stable/) adalah pustaka <i>machine learning</i> sumber terbuka. Pustaka menyediakan berbagai alat untuk penyesuaian model, pemrosesan data di awal, pemilihan dan evaluasi model, serta berbagai utilitas lainnya.

In [ ]:
#Upgrade dependencies
!pip install --upgrade pip
!pip install --upgrade scikit-learn
!pip install --upgrade sagemaker
!pip install --upgrade botocore
!pip install --upgrade awscli

## 1. Membaca set data
([Pergi ke bagian atas](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Anda akan menggunakan pustaka __pandas__ untuk membaca set data. [pandas] (https://pandas.pydata.org/pandas-docs/stable/index.html) adalah pustaka python populer yang digunakan untuk analisis data. Pustaka menyediakan fitur manipulasi, pembersihan, dan perselisihan serta visualisasi data.

In [ ]:
import pandas as pd

df = pd.read_csv('../data/AMAZON-REVIEW-DATA-CLASSIFICATION.csv')

print('The shape of the dataset is:', df.shape)

Lihatlah lima baris pertama dalam set data.

In [ ]:
df.head(5)

Anda dapat mengubah opsi di <i>notebook</i> untuk menampilkan lebih banyak data teks.

In [ ]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)
df.head()

Anda dapat melihat entri tertentu jika diperlukan.

In [ ]:
print(df.loc[[580]])

Ada baiknya mengetahui tipa data apa yang Anda hadapi. Anda dapat menggunakan `dtypes` pada dataframe untuk menampilkan tipe.

In [ ]:
df.dtypes

## 2. Melakukan analisis data eksplorasi
([Pergi ke bagian atas](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Anda sekarang akan melihat distribusi target untuk set data Anda.

In [ ]:
df['isPositive'].value_counts()

Masalah bisnis yang terkait dengan menemukan ulasan negatif (_0_). Namun, penyetelan model untuk pelajar linier secara <i>default </i>diatur untuk menemukan nilai positif (_1_).
 Anda dapat membuat proses ini berjalan lebih lancar dengan mengalihkan nilai negatif (_0_) dan nilai-nilai positif (_1_). Dengan demikian, Anda bisa menyetel model dengan lebih mudah.

In [ ]:
df = df.replace({0:1, 1:0})
df['isPositive'].value_counts()

Periksa jumlah nilai yang hilang:

In [ ]:
df.isna().sum()

Bidang teks memiliki nilai yang hilang. Biasanya, Anda akan memutuskan apa yang harus dilakukan dengan nilai-nilai yang hilang ini. Anda dapat menghapus data atau mengisinya dengan beberapa teks standar. 

## 3. Pemrosesan teks: Menghapus <i>stopword</i> dan <i>stemming</i>
([Pergi ke bagian atas](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Dalam tugas ini, Anda akan menghapus beberapa <i>stopword</i>, dan melakukan <i>stemming </i>pada data teks.
 Anda menormalkan data untuk mengurangi jumlah informasi berbeda yang harus Anda hadapi.

[nltk](https://www.nltk.org/) adalah platform yang populer untuk bekerja dengan data bahasa manusia. Platform menyediakan berbagai antarmuka dan fungsi untuk memproses teks untuk klasifikasi, pemberian token, <i>stemming</i>, pemberian tanda, penguraian, dan penalaran semantik. 

Setelah diimpor, Anda hanya dapat mengunduh fungsionalitas yang Anda butuhkan. Dalam contoh ini, Anda akan menggunakan:

- **<i>punkt</i>** adalah pembuat token kalimat
- **<i>stopword</i>** menyediakan daftar stopword yang dapat Anda gunakan.

In [ ]:
# Install the library and functions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

Anda akan membuat proses untuk menghapus <i>stopword</i> dan membersihkan teks di bagian berikut. Pustaka <i>Natural Language Toolkit</i> (NLTK) menyediakan daftar <i>stopword</i> umum. Anda akan menggunakan daftar, tetapi Anda akan terlebih dulu menghapus beberapa kata dari daftar tersebut. <i>Stopword</i> yang Anda simpan dalam teks berguna untuk menentukan sentimen.

In [ ]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Get a list of stopwords from the NLTK library
stop = stopwords.words('english')

# These words are important for your problem. You don't want to remove them.
excluding = ['against', 'not', 'don', 'don\'t','ain', 'are', 'aren\'t', 'could', 'couldn\'t',
             'did', 'didn\'t', 'does', 'doesn\'t', 'had', 'hadn\'t', 'has', 'hasn\'t', 
             'have', 'haven\'t', 'is', 'isn\'t', 'might', 'mightn\'t', 'must', 'mustn\'t',
             'need', 'needn\'t','should', 'shouldn\'t', 'was', 'wasn\'t', 'were', 
             'weren\'t', 'won\'t', 'would', 'wouldn\'t']

# New stopword list
stopwords = [word for word in stop if word not in excluding]




<i>Stemmer</i> snowball akan memangkas kata-kata. Misalnya, 'berjalan' akan dipangkas ke 'jalan'.

In [ ]:
snow = SnowballStemmer('english')

Anda harus melakukan beberapa tugas normalisasi lainnya pada data. Fungsi berikut akan:

- Mengganti nilai yang hilang dengan string kosong
- Mengonversi teks ke huruf kecil
- Menghapus spasi di depan atau di belakang kalimat
- Menghapus ruang dan tab ekstra
- Menghapus <i>markup</i> HTML

Dalam lingkaran  `for`, kata-kata __NOT__ numeric, lebih panjang dari 2 karakter, dan bukan bagian dari daftar kata henti akan disimpan dan dikembalikan.

In [ ]:
def process_text(texts): 
    final_text_list=[]
    for sent in texts:
        
        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ''
            
        filtered_sentence=[]
        
        sent = sent.lower() # Lowercase 
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:
        
        for w in word_tokenize(sent):
            # Applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stopwords
            if(not w.isnumeric()) and (len(w)>2) and (w not in stopwords):  
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) # Final string of cleaned words
 
        final_text_list.append(final_string)
        
    return final_text_list

## 4. Memisahkan data pelatihan, validasi, dan tes
([Pergi ke bagian atas](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Pada langkah ini, Anda akan membagi set data ke pelatihan (80 persen), validasi (10 persen), dan tes (10 persen) dengan menggunakan fungsi sklearn [__train_test_split()__](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

Data pelatihan akan digunakan untuk melatih model yang kemudian diuji dengan data tes. Set validasi digunakan setelah model telah dilatih untuk memberikan metrik tentang bagaimana model dapat menunjukkan performa pada data sungguhan. 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df[['reviewText', 'summary', 'time', 'log_votes']],
                                                  df['isPositive'],
                                                  test_size=0.20,
                                                  shuffle=True,
                                                  random_state=324
                                                 )

X_val, X_test, y_val, y_test = train_test_split(X_val,
                                                y_val,
                                                test_size=0.5,
                                                shuffle=True,
                                                random_state=324)

Dengan memisahkan set data, kini Anda dapat menjalankan fungsi  `process_text` fungsi yang didefinisikan di atas pada masing-masing fitur teks dalam set pelatihan, tes, dan validasi.

In [ ]:
print('Processing the reviewText fields')
X_train['reviewText'] = process_text(X_train['reviewText'].tolist())
X_val['reviewText'] = process_text(X_val['reviewText'].tolist())
X_test['reviewText'] = process_text(X_test['reviewText'].tolist())

print('Processing the summary fields')
X_train['summary'] = process_text(X_train['summary'].tolist())
X_val['summary'] = process_text(X_val['summary'].tolist())
X_test['summary'] = process_text(X_test['summary'].tolist())

## 5. Memproses data dengan alur dan ColumnTransformer
([Pergi ke bagian atas](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Anda akan sering melakukan banyak tugas pada data sebelum Anda menggunakannya untuk melatih model. Langkah-langkah ini juga harus dilakukan pada setiap data yang digunakan untuk inferensi setelah model di-<i>deploy</i>. Cara yang baik untuk mengatur langkah-langkah ini adalah dengan menentukan _pipeline_. Alur adalah kumpulan tugas pemrosesan yang akan dilakukan pada data. Alur berbeda dapat dibuat untuk memproses bidang yang berbeda. Karena Anda bekerja dengan data teks dan numerik, Anda dapat menentukan alur berikut:

   * Untuk alur fitur numerik, __numerical_processor__ menggunakan MinMaxScaler. (Anda tidak perlu menskalakan fitur saat menggunakan silsilah keputusan, tetapi sebaiknya lihat cara menggunakan lebih banyak perubahan data.) Jika Anda ingin melakukan berbagai tipe pemrosesan pada fitur numerik yang berbeda, Anda sebaiknya membangun alur yang berbeda, seperti yang ditampilkan untuk dua fitur teks.
   * Untuk alur fitur teks __text_processor__ menggunakan  `CountVectorizer()` untuk bidang teks.
   
Persiapan selektif dari fitur set data kemudian disatukan ke dalam ColumnTransformer kolektif, yang akan digunakan dengan dalam alur bersama dengan estimator. Proses ini memastikan bahwa transformasi dilakukan secara otomatis pada data mentah saat Anda menyesuaikan model atau membuat prediksi. (Misalnya, saat Anda mengevaluasi model pada set data validasi melalui validasi silang, atau saat Anda membuat prediksi pada set data tes di masa mendatang.)

In [ ]:
# Grab model features/inputs and target/output
numerical_features = ['time',
                      'log_votes']

text_features = ['summary',
                 'reviewText']

model_features = numerical_features + text_features
model_target = 'isPositive'

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

### COLUMN_TRANSFORMER ###
##########################

# Preprocess the numerical features
numerical_processor = Pipeline([
    ('num_imputer', SimpleImputer(strategy='mean')),
    ('num_scaler', MinMaxScaler()) 
                                ])
# Preprocess 1st text feature
text_processor_0 = Pipeline([
    ('text_vect_0', CountVectorizer(binary=True, max_features=50))
                                ])

# Preprocess 2nd text feature (larger vocabulary)
text_precessor_1 = Pipeline([
    ('text_vect_1', CountVectorizer(binary=True, max_features=150))
                                ])

# Combine all data preprocessors from above (add more, if you choose to define more!)
# For each processor/step specify: a name, the actual process, and finally the features to be processed
data_preprocessor = ColumnTransformer([
    ('numerical_pre', numerical_processor, numerical_features),
    ('text_pre_0', text_processor_0, text_features[0]),
    ('text_pre_1', text_precessor_1, text_features[1])
                                    ]) 

### DATA PREPROCESSING ###
##########################

print('Datasets shapes before processing: ', X_train.shape, X_val.shape, X_test.shape)

X_train = data_preprocessor.fit_transform(X_train).toarray()
X_val = data_preprocessor.transform(X_val).toarray()
X_test = data_preprocessor.transform(X_test).toarray()

print('Datasets shapes after processing: ', X_train.shape, X_val.shape, X_test.shape)

Perhatikan bagaimana jumlah fitur dalam set data berubah dari 4 menjadi 202.

In [ ]:
print(X_train[0])

## 6. Melatih pengklasifikasi dengan algoritma SageMaker bawaan
([Pergi ke bagian atas](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Pada langkah ini, Anda akan memanggil algoritma  `LinearLearner()` Sagemaker dengan opsi berikut:
* __Permissions -__ `role` diatur ke <i>AWS Identity and Access Management</i> (IAM) role dari lingkungan saat ini.
* __Compute power -__ Anda akan menggunakan parameter `train_instance_count` dan parameter `train_instance_type`. Contoh ini menggunakan sumber daya `ml.m4.xlarge` untuk pelatihan. Anda dapat mengubah tipe instans tergantung kebutuhan Anda. (Misalnya, Anda dapat menggunakan GPU untuk jaringan neural.) 
* __Model type -__ `predictor_type` diatur ke __`binary_classifier`__ karena Anda bekerja dengan masalah klasifikasi biner. Anda dapat menggunakan __`multiclass_classifier`__ jika tiga atau lebih kelas terlibat, atau Anda dapat menggunakan __`regressor`__ untuk masalah regresi.


In [ ]:
import sagemaker

# Call the LinearLearner estimator object
linear_classifier = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                           instance_count=1,
                                           instance_type='ml.m4.xlarge',
                                           predictor_type='binary_classifier')

Untuk mengatur bagian pelatihan, validasi, dan tes estimator, Anda dapat menggunakan fungsi `record_set()`  `binary_estimator`. 

In [ ]:
train_records = linear_classifier.record_set(X_train.astype('float32'),
                                            y_train.values.astype('float32'),
                                            channel='train')
val_records = linear_classifier.record_set(X_val.astype('float32'),
                                          y_val.values.astype('float32'),
                                          channel='validation')
test_records = linear_classifier.record_set(X_test.astype('float32'),
                                           y_test.values.astype('float32'),
                                           channel='test')

Fungsi `fit()` menerapkan versi algoritma <i>Stochastic Gradient Descent</i> (SGD), dan Anda mengirim data ke algoritma ini. Log dinonaktifkan dengan `logs=False`. Anda dapat menghapus parameter tersebut untuk melihat detail selengkapnya tentang proses. __Proses ini memerlukan waktu sekitar 3-4 menit pada instans ml.m4.xlarge. __

In [ ]:
linear_classifier.fit([train_records,
                       val_records,
                       test_records],
                       logs=False)

## 7 Mengevaluasi model
([Pergi ke bagian atas](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Anda dapat menggunakan analisis SageMaker untuk mendapatkan beberapa metrik performa (yang Anda pilih) pada set tes. Proses ini tidak mengharuskan Anda untuk men-deploy model. 

Peserta linier menyediakan metrik yang dikomputasi selama pelatihan. Anda dapat menggunakan metrik ini saat menyetel model. Metrik yang tersedia untuk set validasi adalah:

- objective_loss - Untuk masalah klasifikasi biner, ini akan menjadi nilai rata-rata untuk kehilangan logistik untuk setiap <i>epoch</i>
- binary_classification_accuracy - Keakuratan model akhir pada set data, yaitu berapa banyak prediksi yang dibuat oleh model dengan benar

- precision - Menghitung jumlah prediksi kelas positif yang sebenarnya positif
- recall - Menghitung jumlah prediksi kelas positif
- binary_f_beta - Nilai rata-rata metrik presisi dan pemanggilan ulang

Untuk contoh ini, Anda tertarik dengan jumlah prediksi yang benar. Menggunakan metrik **binary_classification_accuracy** tampaknya menjadi pilihan yang tepat.

In [ ]:
sagemaker.analytics.TrainingJobAnalytics(linear_classifier._current_job_name, 
                                         metric_names = ['test:binary_classification_accuracy']
                                        ).dataframe()

Anda seharusnya melihat nilai sekitar 0,85. Nilai Anda mungkin berbeda, tetapi seharusnya di sekitar nilai tersebut. Nilai ini berarti model secara akurat memprediksi jawaban yang benar sebanyak 85%. Tergantung kasus bisnis, Anda mungkin perlu menyetel model lebih lanjut menggunakan pekerjaan penyetelan<i> hyperparameter</i>, atau melakukan beberapa rekayasa fitur lainnya

## 8. Men-<i>deploy</i> model ke titik akhir
([Pergi ke bagian atas](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Dalam bagian terakhir dari latihan ini, Anda akan men-<i>deploy</i> model Anda untuk instans lain pilihan Anda. Anda dapat menggunakan model ini di lingkungan produksi. Titik akhir yang di-<i>deploy</i> dapat digunakan dengan layanan AWS lainnya, seperti AWS Lambda dan Amazon API Gateway. Jika Anda tertarik untuk mempelajari selengkapnya, lihat panduan berikut: [Panggil titik akhir model Amazon SageMaker menggunakan Amazon API Gateway dan AWS Lambda] (https://aws.amazon.com/blogs/machine-learning/call-an-amazon-sagemaker-model-endpoint-using-amazon-api-gateway-and-aws-lambda/).

Untuk men-<i>deploy</i> model, jalankan sel berikut. Anda dapat menggunakan tipe instans berbeda, seperti: _ml.t2.medium_, _ml.c4.xlarge_), dan lainnya. __Proses ini akan memerlukan waktu beberapa saat untuk diselesaikan (sekitar 7-8 menit).__

In [ ]:

linear_classifier_predictor = linear_classifier.deploy(initial_instance_count = 1,
                                                       instance_type = 'ml.c5.large'
                                                      )

## 9. Menguji titik akhir
([Pergi ke bagian atas](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Sekarang setelah titik akhir di-<i>deploy</i>, Anda akan mengirim data tes ke titik akhir dan mendapatkan prediksi dari data.

In [ ]:
import numpy as np

# Get test data in batch size of 25 and make predictions.
prediction_batches = [linear_classifier_predictor.predict(batch)
                      for batch in np.array_split(X_test.astype('float32'), 25)
                     ]

# Get a list of predictions
print([pred.label['score'].float32_tensor.values[0] for pred in prediction_batches[0]])

## 10. Membersihkan model artefak
([Pergi ke bagian atas](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Anda dapat menjalankan berikut untuk menghapus titik akhir setelah Anda selesai menggunakannya. 

**Kiat: ** - Ingat bahwa saat menggunakan akun Anda sendiri, Anda akan dikenai biaya tambahan jika tidak menghapus titik akhir dan sumber daya lainnya.

In [ ]:
linear_classifier_predictor.delete_endpoint()

# Selamat!

Di lab ini, Anda melihat masalah NLP yang sangat sederhana. Dengan menggunakan set data berlabel, Anda menggunakan pembuat token dan <i>encoder </i>sederhana untuk menghasilkan data yang dibutuhkan untuk melatih model pelajar linier.
 Anda kemudian men-<i>deploy</i> model dan melakukan beberapa prediksi. Jika Anda benar-benar melakukan, Anda mungkin akan perlu mendapatkan data dan melabelinya untuk pelatihan. Alternatif mungkin menggunakan algoritma yang telah dilatih sebelumnya atau layanan terkelola. Anda juga mungkin akan menyetel model lebih lanjut menggunakan pekerjaan penyetelan <i>hyperparameter</i>.

Anda telah menyelesaikan lab ini, dan sekarang Anda dapat mengakhirinya dengan mengikuti petunjuk panduan lab tersebut.

©2021, Amazon Web Services, Inc. atau afiliasinya. Hak cipta dilindungi undang-undang. Karya ini tidak boleh direproduksi atau didistribusikan ulang, seluruhnya atau sebagian, tanpa izin tertulis sebelumnya dari Amazon Web Services, Inc. Dilarang menyalin, meminjamkan, atau menjual secara komersial. Semua merek dagang adalah kepemilikan dari pemiliknya.